In [154]:

from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:8b", temperature=0)

res = llm.invoke("user: Hello, how are you?\nassistant:")
print(res)



content="Hello! I'm just a computer program, so I don't have feelings like humans do. But thank you for asking! How can I assist you today?" additional_kwargs={} response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T03:53:36.410359Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10939112458, 'load_duration': 1079611458, 'prompt_eval_count': 20, 'prompt_eval_duration': 8957000000, 'eval_count': 33, 'eval_duration': 900000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-5c9fb268-a627-444a-b993-a09be66b30a3-0' usage_metadata={'input_tokens': 20, 'output_tokens': 33, 'total_tokens': 53}


In [212]:
class Agent:
    def __init__(self, llm, system_msg="You are a helpful assistant"):

        self.llm = llm
        self.system_msg = system_msg
        self.messages = []

        if self.system_msg:
            self.messages.append({"role": "system", "content": self.system_msg})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        prompt = self.make_prompt(self.messages)
        resp = self.llm.invoke(prompt).content
        return resp


    def make_prompt(self, messages):
        return "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        # return "\n".join([f"{m['content']}" for m in messages])



In [213]:
system_msg = """
You are a helpful assistant.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Each output should only begin with Thought, Action, or PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
--------------------------------
""".strip()


In [214]:
agent = Agent(llm, system_msg)
resp = agent("Question: What is 1 + 1?")
print(resp)

Thought: I should run a calculation to find the answer.

Action: calculate: 1 + 1 

PAUSE


In [215]:
print(resp)

Thought: I should run a calculation to find the answer.

Action: calculate: 1 + 1 

PAUSE


In [216]:
def calculate(expression):
    return eval(expression)

calculate("1+1")

2

In [217]:
def average_dog_weight(breed):
    breed = breed.strip().lower()
    if breed in "bulldog":
        return "A Bulldog's average weight is 51 lbs"
    elif breed in "poodle":
        return "A Poodle's average weight is 25 lbs"
    else:
        return "A dog on average weighs 30 lbs"

average_dog_weight("Bulldog")

"A Bulldog's average weight is 51 lbs"

In [218]:
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [219]:
import re
agent = Agent(llm, system_msg)
resp = agent("Question: What is the total combined weight of a Bulldog and a Poodle?")
print(resp)

Thought: I need to find the weight of each dog, so I'll use average_dog_weight for both breeds, then calculate their sum.

Action: average_dog_weight: Bulldog
PAUSE


In [224]:
action_re = re.compile('^Action: (\\w+): (.*)$', re.MULTILINE)
print("#############################")
print("Last response from llm:")
for line in resp.split("\n"):
    print(f"## {line}")

print("#############################")
print("Checking if we need to run an action:")
if action_re.search(resp):
    print("Found [Action:...]")
    action, action_input = action_re.search(resp).groups()
    print(f"Running action: {action} with input: {action_input}")
    observation = known_actions[action](action_input)
    print(f"Observation: {observation}")
    next_prompt = f"\nObservation: {observation}\n"


#############################
Last response from llm:
## Thought: Now that I know the weights of both dogs, I can calculate their total combined weight by adding the two numbers together.
## 
## Action: calculate: 51 + 25
## PAUSE
#############################
Checking if we need to run an action:
Found [Action:...]
Running action: calculate with input: 51 + 25
Observation: 76


In [225]:
print("#############################")
print("Running next prompt:")
for line in next_prompt.split("\n"):
    print(f"## {line}")

resp = agent(next_prompt)
print("#############################")
print("New response from llm:")
for line in resp.split("\n"):
    print(f"## {line}")


#############################
Running next prompt:
## 
## Observation: 76
## 
#############################
New response from llm:
## Thought: The calculation was successful, now I just need to output the result as the final answer.
## 
## Answer: The total combined weight of a Bulldog and a Poodle is 76 lbs.


In [226]:
print("#############################")
print("LLM message history:")
print("#############################")
for line in agent.messages:
    print(f"[{line['role']}]\n{(len(line['role']) + 2) * '#'}\n{line['content']}")



#############################
LLM message history:
#############################
[system]
########
You are a helpful assistant.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Each output should only begin with Thought, Action, or PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation:

In [232]:
# execute llm/tool until we have answer
action_re = re.compile('^Action: (\\w+): (.*)$', re.MULTILINE)
def query(question, max_actions = 10):
    next_prompt = question
    i = 0
    agent = Agent(llm, system_msg)
    while i < max_actions:
        i += 1
        resp = agent(next_prompt)

        if action_re.search(resp):
            action, action_input = action_re.search(resp).groups()
            if action not in known_actions:
                raise ValueError(f"Unknown action: {action}")
            observation = known_actions[action](action_input)
            next_prompt = f"\nObservation: {observation}\n"
        else:
            return resp

            
out = query("What is total combined weight of a Bulldog and a Poodle?")
print(out)

Thought: The calculation was successful, now I just need to output the result as an answer.

Answer: The total combined weight is 76 lbs.
